### generate pool data table

In [2]:
import cmocean as cmo
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import canyon_tools.readout_tools as rout
import canyon_tools.savitzky_golay as sg
import xarray as xr
import matplotlib.gridspec as gspec

In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [4]:
sns.set_context('notebook')
sns.set_style('whitegrid')

In [5]:
sb_Ast = 29 # shelf break z-index Astoria
sb_Bar = 39 # shelf break z-index Barkley

ptr_fileB = '/data/kramosmu/results/TracerExperiments/UPW_10TR_BF4_BAR/01_Bar03/ptracersGlob.nc'
ptr_fileA = '/data/kramosmu/results/TracerExperiments/UPW_10TR_BF2_AST/01_Ast03/ptracersGlob.nc'


#### Get initial concentration at shelf break

In [6]:
tracers = ['Tr01','Tr02','Tr03','Tr04','Tr05','Tr06','Tr07','Tr08','Tr09','Tr10']
labels = ['Linear 01','Salinty 02','Oxygen 03','Nitrate 04','Silicate 05','Phosphate 06','Nitrous Oxide 07','Methane 08',
         'DIC 09', 'Alk 10']
colours = ['#332288','#44AA99','#117733','#999933','#DDCC77','#CC6677','#882255','#AA4499', 'brown', 'tan']
sb_conc_A = np.empty(len(labels))
sb_conc_B = np.empty(len(labels))

for ii, trac in zip(range(len(tracers)),tracers):
    for pfile,sb_array, sb_ind in zip([ptr_fileB, ptr_fileA],[sb_conc_A, sb_conc_B], [sb_Ast, sb_Bar]):
        with Dataset(pfile, 'r') as nbl:
            tr_profile = nbl.variables[trac][0,:,10,180]
            sb_array[ii] = tr_profile[sb_ind]

    

### Table all runs

In [14]:
labels_exp = ['Astoria', 'Ast. ARGO', 'Barkley', 'Bar. Path.']
labels_tra = ['Lin','Sal','Oxy','Nit','Sil','Pho','NiO1E-3','Met1E-3','DIC','Alk']
units = ['(muM)m3/s','PSU','(umol/kg)m3/s','(muM)m3/s','(muM)m3/s','muM)m3/s','(nM)m3/s','(nM)m3/s','(umol/kg)m3/s','(umol/kg)m3/s']

factors = [1,1,1,1,1,1,1,1,1,1]

print ("\t".join(['Run &', 
                  '$A_{pool}/A_{can}$ day 9 &',
                  'max $A_{pool}/A_{can}$) &',
                  '$C$ at day 9 &',
                  'max C &'
                  'max $\Delta C$ (%) &',
                  'mean adv BAC &'
                  'mean adv BAC anom skl'
                 ]))

exp_files = ['saved_calcs/pool_AST.nc',
             'saved_calcs/pool_ARGO.nc',
             'saved_calcs/pool_BAR.nc',
             'saved_calcs/pool_PATH.nc']

exp_files_BAC = ['saved_calcs/BAC_AST.nc',
                 'saved_calcs/BAC_ARGO.nc',
                 'saved_calcs/BAC_BAR.nc',
                 'saved_calcs/BAC_PATH.nc']


can_Area = [1.8E8,1.8E8,8.7E7,8.7E7]
sb_conc = [sb_conc_A, sb_conc_A, sb_conc_B, sb_conc_B]

for tr, unit, tr_lab, factor, ii, trac in zip(tracers, units, labels_tra, factors, range(len(units)), tracers):
    for file,file_bac, lab_exp, can_area, sb_c in zip(exp_files, exp_files_BAC, labels_exp, can_Area, sb_conc):
        with Dataset(file, 'r') as nbl:
            area = nbl.variables['area']
            area_can = nbl.variables['area_can']
            conc = nbl.variables['conc']
            A_day_9 = area[ii,-1]/can_area
            A_max = np.nanmax(area[ii,:])/can_area

            Conc = conc[ii,:]*factor
            C_sb = sb_c[ii]*factor
            C_day_9 = Conc[-1]
            if ii == 2: #oxygen
                C_max = np.nanmin(Conc[:])
            else:
                C_max = np.nanmax(Conc[:])
            
        
        with Dataset(file_bac,'r') as nbl:
            bac = nbl.variables['bac']
            bac_noc = nbl.variables['bac_noc']
            bac_adv = np.nanmean(bac[ii,8:]*factor)
            bac_anom = np.nanmean((bac[ii,8:]-bac_noc[ii,8:])/bac_noc[ii,8:])*100
            
        max_DC = ((C_max-C_sb)/C_sb)*100
        print ("\t".join(['%s %s &' %(lab_exp,tr_lab), 
                 '%1.1f &' %A_day_9,
                 '%1.1f &' %A_max,
                 '%1.1f &' %C_day_9,
                 '%1.1f &' %C_max, 
                 '%1.1f &' %max_DC, 
                 '%1.1f &' %bac_adv, 
                 '%1.1f skl' %bac_anom, 
                          
                ]))

Run &	$A_{pool}/A_{can}$ day 9 &	max $A_{pool}/A_{can}$) &	$C$ at day 9 &	max C &max $\Delta C$ (%) &	mean adv BAC &mean adv BAC anom skl
Astoria Lin &	38.9 &	38.9 &	8.3 &	9.2 &	28.1 &	7.0 &	18.5 skl
Ast. ARGO Lin &	22.8 &	22.8 &	8.3 &	8.7 &	20.7 &	6.7 &	14.8 skl
Barkley Lin &	6.6 &	11.2 &	9.8 &	9.9 &	10.3 &	6.2 &	0.1 skl
Bar. Path. Lin &	23.5 &	23.5 &	10.1 &	10.4 &	15.4 &	6.1 &	0.5 skl
Astoria Sal &	3.8 &	10.7 &	33.9 &	33.9 &	0.1 &	33.7 &	-0.0 skl
Ast. ARGO Sal &	2.7 &	7.1 &	33.9 &	33.9 &	0.1 &	33.7 &	0.0 skl
Barkley Sal &	3.4 &	7.8 &	33.9 &	34.0 &	0.0 &	33.5 &	-0.0 skl
Bar. Path. Sal &	7.9 &	15.0 &	34.0 &	34.0 &	0.1 &	33.5 &	-0.0 skl
Astoria Oxy &	35.3 &	35.3 &	96.2 &	86.7 &	-16.7 &	117.4 &	-5.9 skl
Ast. ARGO Oxy &	20.4 &	20.4 &	95.5 &	90.8 &	-12.7 &	118.7 &	-5.3 skl
Barkley Oxy &	1.9 &	6.5 &	79.4 &	79.4 &	-6.4 &	131.7 &	0.6 skl
Bar. Path. Oxy &	5.7 &	11.0 &	75.7 &	75.7 &	-10.7 &	135.0 &	-0.0 skl
Astoria Nit &	37.0 &	37.0 &	33.7 &	34.9 &	6.9 &	30.7 &	2.4 skl
Ast. ARGO Nit &	21.9 &	21

In [15]:
labels_exp = ['Astoria', 'Ast. ARGO', 'Barkley', 'Bar. Path.']
labels_tra = ['Lin','Sal','Oxy','Nit','Sil','Pho','NiO1E-3','Met1E-3','DIC','Alk']
units = ['(muM)m3/s','PSU','(umol/kg)m3/s','(muM)m3/s','(muM)m3/s','muM)m3/s','(nM)m3/s','(nM)m3/s','(umol/kg)m3/s','(umol/kg)m3/s']

factors = [1,1,1,1,1,1,1,1,1,1]

print ("\t".join(['Run &', 
                  '$A_{pool}/A_{can}$ day 9 &',
                  'max $A_{pool}/A_{can}$) &',
                  '$C$ at day 9 &',
                  'max C &'
                  'max $\Delta C$ (%) &',
                 ]))

exp_files = ['saved_calcs/pool_AST.nc',
             'saved_calcs/pool_ARGO.nc',
             'saved_calcs/pool_BAR.nc',
             'saved_calcs/pool_PATH.nc']

exp_files_BAC = ['saved_calcs/BAC_AST.nc',
                 'saved_calcs/BAC_ARGO.nc',
                 'saved_calcs/BAC_BAR.nc',
                 'saved_calcs/BAC_PATH.nc']


can_Area = [1.8E8,1.8E8,8.7E7,8.7E7]
sb_conc = [sb_conc_A, sb_conc_A, sb_conc_B, sb_conc_B]

for tr, unit, tr_lab, factor, ii, trac in zip(tracers, units, labels_tra, factors, range(len(units)), tracers):
    for file,file_bac, lab_exp, can_area, sb_c in zip(exp_files, exp_files_BAC, labels_exp, can_Area, sb_conc):
        with Dataset(file, 'r') as nbl:
            area = nbl.variables['area']
            area_can = nbl.variables['area_can']
            conc = nbl.variables['conc']
            A_day_9 = area[ii,-1]/can_area
            A_max = np.nanmax(area[ii,:])/can_area

            Conc = conc[ii,:]*factor
            C_sb = sb_c[ii]*factor
            C_day_9 = Conc[-1]
            if ii == 2: #oxygen
                C_max = np.nanmin(Conc[:])
            else:
                C_max = np.nanmax(Conc[:])
                  
        max_DC = ((C_max-C_sb)/C_sb)*100
        print ("\t".join(['%s %s &' %(lab_exp,tr_lab), 
                 '%1.1f &' %A_day_9,
                 '%1.1f &' %A_max,
                 '%1.1f &' %C_day_9,
                 '%1.1f &' %C_max, 
                 '%1.1f skl' %max_DC,         
                ]))

Run &	$A_{pool}/A_{can}$ day 9 &	max $A_{pool}/A_{can}$) &	$C$ at day 9 &	max C &max $\Delta C$ (%) &
Astoria Lin &	38.9 &	38.9 &	8.3 &	9.2 &	28.1 skl
Ast. ARGO Lin &	22.8 &	22.8 &	8.3 &	8.7 &	20.7 skl
Barkley Lin &	6.6 &	11.2 &	9.8 &	9.9 &	10.3 skl
Bar. Path. Lin &	23.5 &	23.5 &	10.1 &	10.4 &	15.4 skl
Astoria Sal &	3.8 &	10.7 &	33.9 &	33.9 &	0.1 skl
Ast. ARGO Sal &	2.7 &	7.1 &	33.9 &	33.9 &	0.1 skl
Barkley Sal &	3.4 &	7.8 &	33.9 &	34.0 &	0.0 skl
Bar. Path. Sal &	7.9 &	15.0 &	34.0 &	34.0 &	0.1 skl
Astoria Oxy &	35.3 &	35.3 &	96.2 &	86.7 &	-16.7 skl
Ast. ARGO Oxy &	20.4 &	20.4 &	95.5 &	90.8 &	-12.7 skl
Barkley Oxy &	1.9 &	6.5 &	79.4 &	79.4 &	-6.4 skl
Bar. Path. Oxy &	5.7 &	11.0 &	75.7 &	75.7 &	-10.7 skl
Astoria Nit &	37.0 &	37.0 &	33.7 &	34.9 &	6.9 skl
Ast. ARGO Nit &	21.9 &	21.9 &	33.8 &	34.3 &	5.2 skl
Barkley Nit &	3.1 &	7.7 &	35.7 &	35.8 &	2.4 skl
Bar. Path. Nit &	9.2 &	14.6 &	36.1 &	36.1 &	3.5 skl
Astoria Sil &	35.3 &	35.3 &	50.2 &	52.9 &	11.1 skl
Ast. ARGO Sil &	19.8 &	19.8 &	50.4 

In [9]:
21.8*2

43.6

In [10]:
13*4


52